In [ ]:
# Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [ ]:
# Chargement des données 
df = pd.read_csv('gapminder_data_graphs.csv')
df.info()

On voit qu'on a 3 colonnes nous permettant de classer les données (pays, continent, année) ainsi que 5 colonnes contenant des valeurs numériques (espérance de vie, Indice de Développement Humain, consommation de CO2, Produit Intérieur Brut et pourcentage de travailleurs dans le secteur tertiaire). 

# Data Cleaning

In [ ]:
# On regarde les valeurs manquantes
plt.figure(figsize = (8,6))
sns.heatmap(df.isnull(), cbar=False , cmap = 'magma')

df.isnull().sum()

In [ ]:
# Affichage des villes et années avec valeurs d'IDH manquantes
df[df['hdi_index'].isna()][['country', 'year']]

Il est cohérent que plusieurs valeurs pour l'indice de développement humain soient manquantes dans le cas de certains pays car il n'était probablement possible de les calculer à l'époque.

In [ ]:
# Affichage des villes et années avec les valeurs de PIB manquantes
df[df['gdp'].isna()][['country', 'year']]

Dans le même cas que précédemment, ces pays manquent de données pour le Produit Brut Intérieur car les données n'étaient probablement pas disponibles à l'époque.

In [ ]:
# Affichage des villes et années avec les valeurs de consommation de CO2 manquantes
df[df['co2_consump'].isna()][['country', 'year']]

Avant 2002, le Timor oriental était une province de l'Indonésie. Nous n'avons donc pas de données pour cette période.

In [ ]:
# Suppression des lignes avec des valeurs manquantes
df.dropna(inplace = True)

# Vérification
df.isnull().sum()

In [ ]:
# On regarde si on a des données dupliquées
print("Nombre de données dupliquées : " , df.duplicated().sum())
df.info()

# Analyse des données

## Analyse descriptive ( moyenne, médiane, écart-type, histogrammes, boxplots, graphiques de lignes)

In [ ]:
# Moyenne des données par pays
df_without_year = df.drop('year', axis = 1)
df_mean = df_without_year.groupby('country').mean(numeric_only=True)
df_mean

In [ ]:
# moyenne la plus élevée pour chaque colonne
print("Pays avec les moyennes les plus haute :\n", df_mean.idxmax())
print("Pays avec les moyenne minimum :\n",df_mean.idxmin())

In [ ]:
# Médianne des données par pays
df_median = df_without_year.groupby('country').median(numeric_only=True)
print(df_median)


In [ ]:
# histogramme comparaison des varibles par continent
df_mean = df.groupby(['country', 'continent']).mean(numeric_only=True)
fig, ax = plt.subplots(5, 1, figsize = (10, 20))

sns.histplot(data = df_mean, x = 'life_exp', hue = 'continent', ax = ax[0], kde = True)
sns.histplot(data = df_mean, x = 'hdi_index', hue = 'continent', ax = ax[1], kde = True)
sns.histplot(data = df_mean, x = 'co2_consump', hue = 'continent', ax = ax[2], kde = True)
sns.histplot(data = df_mean, x = 'gdp', hue = 'continent', ax = ax[3], kde = True)
sns.histplot(data = df_mean, x = 'services', hue = 'continent', ax = ax[4], kde = True) 
plt.show()

In [ ]:
# boxplot comparaison des varibles par continent
df_mean = df.groupby(['country', 'continent']).mean(numeric_only=True).reset_index()

fig, ax = plt.subplots(5, 1, figsize = (10, 20))
sns.boxplot(data = df_mean, x = 'services', y = 'continent', ax = ax[0])
sns.boxplot(data = df_mean, x = 'life_exp', y = 'continent', ax = ax[1])
sns.boxplot(data = df_mean, x = 'hdi_index', y = 'continent', ax = ax[2])
sns.boxplot(data = df_mean, x = 'gdp', y = 'continent', ax = ax[3])
sns.boxplot(data = df_mean, x = 'co2_consump', y = 'continent', ax = ax[4])
plt.show()

In [ ]:
# graphique en barre de la consommation de CO2 par continent
df_mean = df.groupby(['country', 'continent']).mean(numeric_only=True).reset_index()

fig = px.bar(df_mean, x = 'continent', y = 'co2_consump', color = 'country', title = 'Consommation de CO2 par continent')
fig.show()

In [ ]:
fig = px.bar(df_mean, x = 'continent', y = 'life_exp', color = 'country')

fig.show()

In [ ]:
fig = px.bar(df_mean, x = 'continent', y = 'services', color = 'country')
fig.show()

In [ ]:
fig = px.bar(df_mean, x = 'continent', y = 'hdi_index', color = 'country')
fig.show()

In [ ]:
fig = px.bar(df_mean, x = 'continent', y = 'gdp', color = 'country')
fig.show()

## Corrélation entre les colonnes

On va regarder quelles sont les colonnes qui sont corrélées entre elles pour voir si on peut en déduire quelque chose.

In [ ]:
# On regarde les colonnes corrélées et on affiche dans une heatmap
df_num = df.select_dtypes(include = ['float64'])
correlation = df_num.corr()
plt.figure(figsize = (8,6))
sns.heatmap(correlation, annot = True, cmap = 'coolwarm')

On remarque que les valeurs des colonnes sont plutôt liées mais on voit notamment une forte corrélation entre l'IDH et l'espérance de vie ce qui est logique car l'IDH est calculé en prenant en compte l'espérance de vie. On observe également une corrélation importante entre l'IDH et le pourcentage de travailleurs dans le secteur tertiaire qui peut s'expliquer par le fait que les pays avec un IDH élevé ont souvent une économie tertiaire développée comme avec le tourisme par exemple.

In [ ]:
# Affichage des relations entre les colonnes
liste_colonnes = ["continent", "life_exp", "hdi_index", "co2_consump", "gdp", "services"]
sns.pairplot(data=df.loc[:, liste_colonnes], hue="continent")

Ces différents graphiques illustrent bien les corrélations plus ou moins fortes entre les différentes colonnes. Les graphiques avec des points désordonnés et dispersés montrent une faible corrélation entre les colonnes tandis que les graphiques avec des points qui suivent approximativement une droite montrent une corrélation plus forte.

On peut aussi essayer de regarder la corrélation entre les variables numériques et les différents continents pour voir si on peut en déduire quelque chose.

In [ ]:
# On ajoute une variable pour chaque continent avec un booléen pour savoir si le pays est dans ce continent
df_variables_continents = pd.get_dummies(df, columns = ['continent'])
df_variables_continents = df_variables_continents.select_dtypes(include = ['float64', 'bool'])
# Corrélation entre les variables
correlation = df_variables_continents.corr()
plt.figure(figsize = (8,6))
sns.heatmap(correlation, annot = True, cmap = 'coolwarm')

On peut s'apercevoir ici que les coefficients de corrélation entre nos variables et les continents sont très différents selon les continents. Ceux-ci sont vont de 0.23 à 0.53 pour l'Europe tandis qu'ils sont situés entre -0.69 et -0.35 pour l'Afrique. Il est possible d'expliquer cela par le fait que l'Europe est un continent développé et possède donc des valeurs élevées pour l'IDH, le PIB ou l'espérance de vie alors que l'Afrique est un continent en développement et possède donc des valeurs plus faibles pour ces mêmes variables.

## Clustering

## Transformation de données

## Analyse temporelles et spatiales

In [ ]:
# Création d'un graphique qui montre l'évolution de l'IDH et du % de services en fonction de l'année
df_complet = pd.read_csv('gapminder_data_graphs.csv')
graphique = px.scatter(df_complet, x = "hdi_index",  y = "services", color = "continent", animation_frame="year", range_x=[0,1], range_y= [0, 100])
graphique.show()

In [ ]:
graphique = px.scatter(df_complet, x = "hdi_index",  y = "life_exp", color = "continent", animation_frame="year", range_x=[0,1], range_y= [0, 100])
graphique.show()

In [ ]:

# Moyennes par année et continent
data_temporel = df.groupby(['year', 'continent']).mean(numeric_only=True).reset_index()
fig, ax = plt.subplots(5, 1, figsize=(12, 20))
sns.lineplot(data=data_temporel, x='year', y='life_exp', hue='continent', ax= ax[0], marker='o')
sns.lineplot(data=data_temporel, x='year', y='hdi_index', hue='continent', ax= ax[1], marker='o')
sns.lineplot(data=data_temporel, x='year', y='co2_consump', hue='continent', ax= ax[2], marker='o')
sns.lineplot(data=data_temporel, x='year', y='gdp', hue='continent', ax= ax[3], marker='o')
sns.lineplot(data=data_temporel, x='year', y='services', hue='continent', ax= ax[4], marker='o')

## Détection d'anomalies

In [ ]:
window_size = 4  # Taille de la fenêtre pour la moyenne mobile
deviation_threshold = 2  # Seuil pour la déviation

# Fonction pour calculer la moyenne mobile et détecter les anomalies
def detect_anomalies(group):
    group['moving_avg'] = group['life_exp'].rolling(window=window_size, center=True).mean()
    group['deviation'] = group['life_exp'] - group['moving_avg']
    group['anomaly'] = (abs(group['deviation']) > deviation_threshold)
    return group

# Appliquer la fonction à chaque pays
data = df.groupby('country').apply(detect_anomalies)

anomalies = data[data['anomaly']]
print("Anomalies détectées :")
anomalies[['country', 'year', 'life_exp', 'moving_avg', 'deviation']]


In [ ]:
window_size = 4  # Taille de la fenêtre pour la moyenne mobile
deviation_threshold = 3  # Seuil pour la déviation

# Fonction pour calculer la moyenne mobile et détecter les anomalies
def detect_anomalies(group):
    group['moving_avg'] = group['co2_consump'].rolling(window=window_size, center=True).mean()
    group['deviation'] = group['co2_consump'] - group['moving_avg']
    group['anomaly'] =  (abs(group['deviation']) > deviation_threshold)
    
    return group

# Appliquer la fonction à chaque pays
data = df.groupby('country').apply(detect_anomalies)

anomalies = data[data['anomaly']]
print("Anomalies détectées :")
anomalies[['country', 'year', 'co2_consump', 'moving_avg', 'deviation']]

In [ ]:
window_size = 4  # Taille de la fenêtre pour la moyenne mobile
deviation_threshold = 2000  # Seuil pour la déviation

# Fonction pour calculer la moyenne mobile et détecter les anomalies
def detect_anomalies(group):
    group['moving_avg'] = group['gdp'].rolling(window=window_size, center=True).mean()
    group['deviation'] = group['gdp'] - group['moving_avg']
    group['anomaly'] =  (abs(group['deviation']) > deviation_threshold)
    
    return group

# Appliquer la fonction à chaque pays
data = df.groupby('country').apply(detect_anomalies)

anomalies = data[data['anomaly']]
print("Anomalies détectées :")
anomalies[['country', 'year', 'gdp', 'moving_avg', 'deviation']]

In [ ]:
window_size = 4  # Taille de la fenêtre pour la moyenne mobile
deviation_threshold = 2  # Seuil pour la déviation

# Fonction pour calculer la moyenne mobile et détecter les anomalies
def detect_anomalies(group):
    group['moving_avg'] = group['services'].rolling(window=window_size, center=True).mean()
    group['deviation'] = group['services'] - group['moving_avg']
    group['anomaly'] =  (abs(group['deviation']) > deviation_threshold)
    
    return group

# Appliquer la fonction à chaque pays
data = df.groupby('country').apply(detect_anomalies)

anomalies = data[data['anomaly']]
print("Anomalies détectées :")
anomalies[['country', 'year', 'services', 'moving_avg', 'deviation']]